In [1]:
import pandas as pd

In [4]:
base_path = "./dataset/"
melbourne_data = pd.read_csv(base_path + "melbourne-housing-snapshot.csv")
melbourne_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [5]:
y = melbourne_data.Price

In [8]:
ex_obj = melbourne_data.select_dtypes(exclude="object")
ex_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          13580 non-null  int64  
 1   Price          13580 non-null  float64
 2   Distance       13580 non-null  float64
 3   Postcode       13580 non-null  float64
 4   Bedroom2       13580 non-null  float64
 5   Bathroom       13580 non-null  float64
 6   Car            13518 non-null  float64
 7   Landsize       13580 non-null  float64
 8   BuildingArea   7130 non-null   float64
 9   YearBuilt      8205 non-null   float64
 10  Lattitude      13580 non-null  float64
 11  Longtitude     13580 non-null  float64
 12  Propertycount  13580 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 1.3 MB


In [9]:
X = ex_obj

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [32]:
def score_dataset(missings: list, train_X, val_X, train_y, val_y):
    # I didn't naming it.
    model = RandomForestRegressor(random_state=42)
    model.fit(train_X, train_y)
    pred = model.predict(val_X)
    mae = mean_absolute_error(val_y, pred)
    return mae

In [41]:
score_dataset([], train_X.dropna(axis=1), val_X.dropna(axis=1), train_y, val_y)

341.82539027982307

In [56]:
cols_contain_miss = [col for col in train_X.columns if train_X[col].isna().any()]
cols_contain_miss

['Car', 'BuildingArea', 'YearBuilt']

In [59]:
without_miss_cols = train_X.drop(cols_contain_miss, axis=1)
without_miss_cols

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12796,4,1480000.0,14.2,3149.0,4.0,2.0,695.0,-37.86127,145.14271,13366.0
9642,3,1530000.0,14.2,3149.0,3.0,1.0,810.0,-37.86838,145.14664,13366.0
3207,2,690000.0,4.6,3122.0,2.0,1.0,82.0,-37.81800,145.02680,11308.0
1698,2,570000.0,3.2,3054.0,2.0,1.0,0.0,-37.79020,144.97000,3106.0
761,4,1041000.0,13.0,3204.0,4.0,2.0,292.0,-37.91480,145.02430,6795.0
...,...,...,...,...,...,...,...,...,...,...
5191,3,851500.0,11.2,3073.0,3.0,1.0,0.0,-37.72170,144.99540,21650.0
13418,2,1400000.0,3.6,3068.0,2.0,1.0,164.0,-37.77796,144.98374,6244.0
5390,3,975000.0,2.6,3121.0,3.0,1.0,383.0,-37.82480,144.99760,14949.0
860,2,722000.0,13.9,3165.0,2.0,2.0,201.0,-37.92930,145.06570,10969.0


In [62]:
train_X_red = train_X.drop(cols_contain_miss, axis=1)
val_X_red = val_X.drop(cols_contain_miss, axis=1)

In [63]:
print(f"MAE from approach 1. (Drop columns contain missing):")
score_dataset([], train_X_red, val_X_red, train_y, val_y)

MAE from approach 1. (Drop columns contain missing):


341.82539027982307

In [64]:
from sklearn.impute import SimpleImputer

In [65]:
s_imputer = SimpleImputer()
train_X_imputed = pd.DataFrame(s_imputer.fit_transform(train_X))
val_X_imputed = pd.DataFrame(s_imputer.transform(val_X))

In [68]:
train_X.shape, train_X_imputed.shape

((10864, 13), (10864, 13))

In [69]:
val_X.shape, val_X_imputed.shape

((2716, 13), (2716, 13))